In [1]:
import logging

import config
import lightgbm as lgb
import polars as pl
import xgboost as xgb
from catboost import CatBoostClassifier, CatBoostRegressor
from lifelines import BreslowFlemingHarringtonFitter, KaplanMeierFitter

from src.customs.fold import add_kfold
from src.customs.metrics import CatBoostMetric, Metric, ROCAUCMetric
from src.feature.tabular import AggregateEncoder, OrdinalEncoder, RawEncoder
from src.model.sklearn_like import (
    CatBoostClassifierWrapper,
    CatBoostRegressorWrapper,
    LightGBMWapper,
    XGBoostRegressorWrapper,
)
from src.trainer.tabular.simple import single_inference_fn, single_inference_fn_v2, single_train_fn

logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

In [2]:
class CreateTargetFn:
    def __init__(self, time_col: str, event_col: str):
        self.time_col = time_col
        self.event_col = event_col
        self.target_cols = [
            "target_kmf",
            "target_kmf_oof",
            "target_xgboost_cox",
            "target_bfhf",
        ]

    def __call__(self, df: pl.DataFrame) -> pl.DataFrame:
        # add kmf target by race_group
        train_df = df.filter(pl.col(config.DATASET_COL) == "TRAIN")
        target_df = df.clone()

        # add kmf target
        kmf = KaplanMeierFitter()
        kmf.fit(train_df[self.time_col], event_observed=train_df[self.event_col])
        target_df = target_df.with_columns(
            pl.Series("target_kmf", kmf.survival_function_at_times(target_df[self.time_col])),
        )

        # add oof kmf target
        fold_df = add_kfold(target_df, n_splits=5, random_state=config.SEED, fold_col="_fold")
        oof_df = pl.DataFrame()
        for fold in range(config.N_SPLITS):
            _tr_df = fold_df.filter(pl.col("_fold") != fold)
            _va_df = fold_df.filter(pl.col("_fold") == fold)
            kmf = KaplanMeierFitter()
            kmf.fit(_tr_df[self.time_col], event_observed=_tr_df[self.event_col])
            oof_df = pl.concat(
                [
                    oof_df,
                    (
                        _va_df.select(
                            pl.col(config.ID_COL),
                            pl.Series("target_kmf_oof", kmf.survival_function_at_times(_va_df[self.time_col])),
                        )
                    ),
                ],
                how="diagonal_relaxed",
            )
        target_df = target_df.join(oof_df, on=config.ID_COL, how="left")

        # add xgboost cox target
        target_df = target_df.with_columns(
            pl.when(target_df[self.event_col] == 0)
            .then(-1 * target_df[self.time_col])
            .otherwise(target_df[self.time_col])
            .alias("target_xgboost_cox"),
        )
        # add bfhf target
        bfhf = BreslowFlemingHarringtonFitter()
        bfhf.fit(train_df[self.time_col], event_observed=train_df[self.event_col])
        target_df = target_df.with_columns(
            pl.Series("target_bfhf", bfhf.survival_function_at_times(target_df[self.time_col])),
        )

        return target_df


def load_data(valid_ratio: float | None = None) -> pl.DataFrame:
    raw_train_df = pl.read_csv(config.COMP_DATASET_DIR / "train.csv").with_columns(
        pl.lit("TRAIN").alias(config.DATASET_COL),
        pl.lit(-1).alias(config.FOLD_COL),
    )
    if valid_ratio is not None:
        from sklearn.model_selection import train_test_split

        _tr_df, _va_df = train_test_split(raw_train_df, test_size=0.2, random_state=config.SEED)
        _va_df = _va_df.with_columns(
            pl.lit("VALID").alias(config.DATASET_COL),
        )
        raw_train_df = pl.concat([_tr_df, _va_df], how="diagonal_relaxed")

    create_target_fn = CreateTargetFn(time_col=config.SURVIVAL_TIME_COL, event_col=config.EVENT_COL)
    raw_train_df = create_target_fn(raw_train_df)

    raw_test_df = pl.read_csv(config.COMP_DATASET_DIR / "test.csv").with_columns(
        pl.lit("TEST").alias(config.DATASET_COL)
    )

    config.META_COLS = sorted(list(set(config.META_COLS + create_target_fn.target_cols)))

    train_test_df = pl.concat([raw_train_df, raw_test_df], how="diagonal_relaxed").sort(
        config.DATASET_COL, config.ID_COL, descending=[True, False]
    )
    return train_test_df


def fe(
    train_test_df: pl.DataFrame,
    output_dataset: str = "TRAIN",
) -> pl.DataFrame:
    encoders = [
        RawEncoder(columns=config.META_COLS, prefix=""),
        RawEncoder(
            columns=(
                [
                    *config.NUMERICAL_COLS,
                ]
            ),
            prefix=f"{config.FEATURE_PREFIX}n_",
        ),
        OrdinalEncoder(
            columns=(
                [
                    *config.CATEGORICAL_COLS,
                ]
            ),
            prefix=f"{config.FEATURE_PREFIX}c_",
        ),
    ]

    for encoder in encoders:
        encoder.fit(train_test_df.filter(pl.col(config.DATASET_COL) == "TRAIN"))

    features_df = pl.concat(
        [
            encoder.transform(
                train_test_df.filter(pl.col(config.DATASET_COL) == output_dataset),
            )
            for encoder in encoders
        ],
        how="horizontal",
    )
    return features_df

In [ ]:
train_test_df = load_data(valid_ratio=config.VALID_RATIO)
features_df = fe(train_test_df)

feature_names = sorted([x for x in features_df.columns if x.startswith(config.FEATURE_PREFIX)])
cat_features = [x for x in feature_names if x.startswith(f"{config.FEATURE_PREFIX}c_")]


logger.info(f"# of features: {len(feature_names)}")
logger.info(f"# of cat_features: {len(cat_features)}")

In [ ]:
train_features_df = add_kfold(
    features_df,
    n_splits=4,
    random_state=config.SEED,
    fold_col=f"{config.FOLD_COL}_nested",
)


all_result_df = pl.DataFrame()
for i_fold in train_features_df[f"{config.FOLD_COL}_nested"].unique().to_list():
    logger.info(f"Outer Fold: {i_fold}")
    tr_df = train_features_df.filter(pl.col(f"{config.FOLD_COL}_nested") != i_fold)
    va_df = train_features_df.filter(pl.col(f"{config.FOLD_COL}_nested") == i_fold)
    print(tr_df.shape, va_df.shape)
    # -------------------
    # cat
    _, _, trained_models = single_train_fn(
        model=CatBoostClassifierWrapper(
            name=f"cat_{i_fold}",
            model=CatBoostClassifier(
                loss_function="Logloss",
                learning_rate=0.1,
                n_estimators=10000,
                early_stopping_rounds=500,
                # eval_metric=CatBoostMetric(),
                verbose=100,
                # subsample=0.5,
                colsample_bylevel=0.2,
                random_state=config.SEED,
                # task_type="GPU",
            ),
            multi_output=False,
            feature_names=feature_names,
            cat_features=cat_features,
        ),
        features_df=add_kfold(
            tr_df,
            n_splits=4,
            random_state=config.SEED,
            fold_col=config.FOLD_COL,
        ),
        feature_cols=feature_names,
        target_col=config.EVENT_COL,
        fold_col=config.FOLD_COL,
        meta_cols=config.META_COLS,
        # weight_col="sample_weight",
        out_dir=config.OUTPUT_DIR,
        train_folds=None,
        eval_fn=ROCAUCMetric(),
        overwrite=False,
        # use_eval_metric_extra_va_df=True,
    )
    va_result_df = single_inference_fn_v2(
        models=trained_models,
        features_df=va_df,
        feature_names=feature_names,
    )
    all_result_df = pl.concat(
        [
            all_result_df,
            va_result_df.select(
                pl.col(config.ID_COL),
                pl.col("pred"),
            ),
        ],
        how="diagonal_relaxed",
    )
    # -------------------
    # lgb
    _, _, trained_models = single_train_fn(
        model=LightGBMWapper(
            name=f"lgb_{i_fold}",
            model=lgb.LGBMModel(
                objective="binary",
                boosting="gbdt",
                n_estimators=10000,
                learning_rate=0.02,
                num_leaves=31,
                colsample_bytree=0.2,
                subsample=0.5,
                importance_type="gain",
                random_state=config.SEED,
            ),
            fit_params={
                "callbacks": [
                    lgb.early_stopping(500, first_metric_only=True),
                    lgb.log_evaluation(period=100),
                ],
                "categorical_feature": cat_features,
                "feature_name": feature_names,
                # "eval_metric": "auc",
            },
        ),
        features_df=add_kfold(
            tr_df,
            n_splits=4,
            random_state=config.SEED,
            fold_col=config.FOLD_COL,
        ),
        feature_cols=feature_names,
        target_col=config.EVENT_COL,
        fold_col=config.FOLD_COL,
        meta_cols=config.META_COLS,
        # weight_col="sample_weight",
        out_dir=config.OUTPUT_DIR,
        train_folds=None,
        eval_fn=ROCAUCMetric(),
        overwrite=False,
        # use_eval_metric_extra_va_df=True,
    )
    va_result_df = single_inference_fn_v2(
        models=trained_models,
        features_df=va_df,
        feature_names=feature_names,
    )
    #

    all_result_df = pl.concat(
        [
            all_result_df,
            va_result_df.select(
                pl.col(config.ID_COL),
                pl.col("pred"),
            ),
        ],
        how="diagonal_relaxed",
    )

all_result_df = all_result_df.group_by(config.ID_COL).agg(pl.mean("pred").alias("pred"))

# INFO:src.trainer.tabular.simple:✅ Final Metric: 0.6897873116621929
# INFO:src.trainer.tabular.simple:✅ Final Metric: 0.6898640681195222


va_result_df_1st = all_result_df.join(train_features_df.select(config.META_COLS), on=config.ID_COL, how="left")
logger.info(f"{ROCAUCMetric()._name}: {ROCAUCMetric()(va_result_df_1st)}")

In [ ]:
lower_bound = 0.0
scaling_factor_pos = (
    va_result_df_1st.filter(pl.col(config.EVENT_COL) == 0)
    .select(pl.col("pred").log().min() / (0 - pl.col("target_kmf").min()))["pred"]
    .to_numpy()[0]
)
scaling_factor_neg = (
    va_result_df_1st.filter(pl.col(config.EVENT_COL) == 0)
    .select(pl.col("pred").log().min() / (0 - pl.col("target_kmf").min()))["pred"]
    .to_numpy()[0]
)

logger.info(f"scaling_factor: {scaling_factor_pos}, {scaling_factor_neg}")
_df = va_result_df_1st.select(
    pl.col(config.ID_COL),
    pl.col("pred"),
    pl.when(pl.col(config.EVENT_COL) == 0)
    .then(pl.col("pred").log() / scaling_factor_pos + pl.col("target_kmf"))
    .otherwise(pl.col("pred").log() / scaling_factor_neg + pl.col("target_kmf"))
    .alias("target"),
)

features_df_2nd = features_df.join(_df, on=config.ID_COL, how="left")

# features_df = features_df.drop("target")

In [57]:
import numpy as np
from scipy.stats import rankdata


def transform_y(y, event, w):
    transformed = np.full(len(y), np.nan)

    transformed[event == 1] = y[event == 1]
    transformed[event == 0] = y[event == 0]

    # ranking
    # transformed = rankdata(transformed)
    # 0~1 normalization
    # transformed = (transformed - transformed.min()) / (transformed.max() - transformed.min())
    return transformed


t = transform_y(
    event=features_df_2nd[config.EVENT_COL].to_numpy(),
    w=(features_df_2nd["pred"].to_numpy()),
    y=features_df_2nd["target"].to_numpy(),
)
w = features_df_2nd["pred"].to_numpy().copy()
w[features_df_2nd[config.EVENT_COL].to_numpy() == 1] = 1
w[features_df_2nd[config.EVENT_COL].to_numpy() == 0] = 0.5
features_df_2nd = features_df_2nd.with_columns(pl.Series("custom_target", t), pl.Series("custom_weight", w))

In [ ]:
train_features_df = add_kfold(
    features_df_2nd,
    n_splits=config.N_SPLITS,
    random_state=config.SEED,
    fold_col=config.FOLD_COL,
)

va_result_df, va_scores, trained_models = single_train_fn(
    model=CatBoostRegressorWrapper(
        name="cat2",
        model=CatBoostRegressor(
            loss_function="RMSE",
            learning_rate=0.1,
            n_estimators=50000,
            early_stopping_rounds=500,
            eval_metric=CatBoostMetric(),
            verbose=100,
            # subsample=0.5,
            colsample_bylevel=0.2,
            random_state=config.SEED,
        ),
        multi_output=False,
        feature_names=feature_names,
        cat_features=cat_features,
    ),
    features_df=train_features_df,
    feature_cols=feature_names,
    target_col="custom_target",
    fold_col=config.FOLD_COL,
    meta_cols=config.META_COLS,
    weight_col="custom_weight",
    out_dir=config.OUTPUT_DIR,
    train_folds=None,
    eval_fn=Metric(),
    overwrite=True,
    use_eval_metric_extra_va_df=True,
)

# bestTest = 0.6823613885
# bestIteration = 5010
# 1000:	learn: 0.0000000	test: 0.6729263	best: 0.6730469 (986)	total: 20s	remaining: 2m 59s


In [ ]:
if "VALID" in train_test_df[config.DATASET_COL].unique():
    ext_va_features_df = fe(train_test_df, output_dataset="VALID")
    ext_va_result_df = single_inference_fn(
        model=CatBoostRegressorWrapper(name="cat2", feature_names=feature_names, cat_features=cat_features),
        features_df=ext_va_features_df,
        feature_names=feature_names,
        model_dir=config.ARTIFACT_EXP_DIR(),
        inference_folds=list(range(config.N_SPLITS)),
        out_dir=config.OUTPUT_DIR,
    )
    print(ext_va_result_df["pred"].to_list())
    print(Metric(prediction_label="pred")(ext_va_result_df))

    # 0.6850974473819677 -> 0.6823781299242823


In [ ]:
# debug test
test_features_df = fe(train_test_df, output_dataset="TEST")
te_result_df = single_inference_fn(
    model=CatBoostRegressorWrapper(name="cat2", feature_names=feature_names, cat_features=cat_features),
    features_df=test_features_df,
    feature_names=feature_names,
    model_dir=config.ARTIFACT_EXP_DIR(),
    inference_folds=list(range(config.N_SPLITS)),
    out_dir=config.OUTPUT_DIR,
)
print(te_result_df["pred"].to_list())